In [2]:
import pandas as pd

df = pd.read_csv('/Users/mohammednihal/Desktop/XAI/data/Global Superstore.csv',encoding='ISO-8859-1')
df = df.drop([col  for col in df.columns if 'Unnamed' in col], axis=1)

print(df.head())

print("DataFrame shape :" , df.shape)
print("\nDataFrame  head:")
print(df.head())
print("\nDataFrame columns:", df.columns.tolist())


   Row ID         Order ID  Order Date  ...    Profit Shipping Cost Order Priority
0   32298   CA-2012-124891  31-07-2012  ...  762.1845        933.57       Critical
1   26341    IN-2013-77878  05-02-2013  ... -288.7650        923.63       Critical
2   25330    IN-2013-71249  17-10-2013  ...  919.9710        915.49         Medium
3   13524  ES-2013-1579342  28-01-2013  ...  -96.5400        910.16         Medium
4   47221     SG-2013-4320  05-11-2013  ...  311.5200        903.04       Critical

[5 rows x 24 columns]
DataFrame shape : (51290, 24)

DataFrame  head:
   Row ID         Order ID  Order Date  ...    Profit Shipping Cost Order Priority
0   32298   CA-2012-124891  31-07-2012  ...  762.1845        933.57       Critical
1   26341    IN-2013-77878  05-02-2013  ... -288.7650        923.63       Critical
2   25330    IN-2013-71249  17-10-2013  ...  919.9710        915.49         Medium
3   13524  ES-2013-1579342  28-01-2013  ...  -96.5400        910.16         Medium
4   47221     SG

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta


def data_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    df_clean = df.copy()

    # Convert datetime columns
    datetime_columns = ['Order Date', 'Ship Date']
    for col in datetime_columns:
        if col in df_clean.columns:
            df_clean[col] = pd.to_datetime(df_clean[col], dayfirst=True, errors='coerce')

    # Remove exact duplicates
    df_clean = df_clean.drop_duplicates()

    # Remove potential duplicates based on Customer ID, Sales, and Profit
    df_clean = df_clean.drop_duplicates(subset=['Customer ID', 'Sales', 'Profit'], keep='first')

    # Fill missing numerical values with median
    numeric_columns = df_clean.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

    # Fill missing categorical values with mode
    categorical_columns = df_clean.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        if not df_clean[col].mode().empty:
            df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

    # Handle outliers using IQR
    def handle_outliers(series):
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        return series.clip(lower=lower_bound, upper=upper_bound)

    numeric_cols_for_outliers = ['Sales', 'Profit', 'Quantity', 'Discount', 'Shipping Cost']
    for col in numeric_cols_for_outliers:
        if col in df_clean.columns:
            df_clean[col] = handle_outliers(df_clean[col])

    # Clip specific ranges
    df_clean['Profit'] = df_clean['Profit'].clip(lower=-1500, upper=10000)
    df_clean['Quantity'] = df_clean['Quantity'].clip(lower=1)
    df_clean['Discount'] = df_clean['Discount'].clip(0, 1)
    df_clean['Shipping Cost'] = df_clean['Shipping Cost'].clip(lower=0, upper=1000)

    # Calculate Unit Price safely
    discount_safe = (1 - df_clean['Discount']).replace(0, np.nan)
    df_clean['Unit_Price'] = (df_clean['Sales'] - df_clean['Shipping Cost'] - df_clean['Profit']) / (df_clean['Quantity'] * discount_safe)

    # Standardize categorical variables
    standardize_cols = ['Customer Name', 'City', 'State', 'Product Name', 'Country', 'Order Priority', 'Region', 'Market']
    for col in standardize_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].str.lower()

    # Remove any remaining NaNs
    df_clean = df_clean.dropna()

    # Sort by Order Date
    if 'Order Date' in df_clean.columns:
        df_clean = df_clean.sort_values(by='Order Date', ascending=True)

    df_clean.reset_index(drop=True, inplace=True)

    print(df_clean.head())

    # Optional: Save to CSV
    df_clean.to_csv('cleaned_data.csv', index=False)

    return df_clean


def Initial_col_drop(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_drop = [
        'Row ID', 'Order ID', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name',
        'City', 'State', 'Country', 'Postal Code', 'Market', 'Region', 'Product ID',
        'Product Name', 'Year-Month', 'Sub-Category'
    ]
    return df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')


def main(df: pd.DataFrame):
    
    try:
        # Initial data overview
        print("\nInitial Data Overview:")
        print(f"Initial shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        print("\nMissing Values Overview:")
        print(df.isnull().sum())

        # Cleaning data
        print("\nCleaning Data...")
        cleaned_df = data_cleaning(df)
        print(f"Data cleaned. Remaining rows: {len(cleaned_df)}")

        # Dropping unnecessary columns
        cleaned_df = Initial_col_drop(cleaned_df)
        print("Columns Dropped Successfully")
        print(f"Remaining columns: {cleaned_df.columns.tolist()}")

        return cleaned_df

    except Exception as e:
        print(f"Error in Data Cleaning Pipeline: {str(e)}")
        return None


if __name__ == "__main__":
    
    try:
        df = df  
        cleaned_df = main(df)

        if cleaned_df is not None:
            print("\nData Cleaning Pipeline Executed Successfully!")
        else:
            print("\nData Cleaning Pipeline Failed.")
    except FileNotFoundError:
        print("Data file not found. Please ensure 'your_data_file.csv' exists.")



Initial Data Overview:
Initial shape: (51290, 24)
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'City', 'State', 'Country', 'Postal Code', 'Market', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost', 'Order Priority']

Missing Values Overview:
Row ID                0
Order ID              0
Order Date            0
Ship Date             0
Ship Mode             0
Customer ID           0
Customer Name         0
Segment               0
City                  0
State                 0
Country               0
Postal Code       41296
Market                0
Region                0
Product ID            0
Category              0
Sub-Category          0
Product Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping Cost         0
Order Priority        0
dtype: int64

Cleaning Data...
  

In [4]:
print(cleaned_df['Category'].head(10))

0    Office Supplies
1    Office Supplies
2    Office Supplies
3    Office Supplies
4          Furniture
5    Office Supplies
6         Technology
7         Technology
8          Furniture
9    Office Supplies
Name: Category, dtype: object


In [5]:
column_name = 'Order Date'
print(cleaned_df[column_name].head())

0   2011-01-01
1   2011-01-01
2   2011-01-01
3   2011-01-01
4   2011-01-01
Name: Order Date, dtype: datetime64[ns]


In [6]:
import pandas as pd
import numpy as np

def check_missing_values(df, function_name):
    """Helper function to check and print missing values after feature creation."""
    print(f"Missing values in {function_name}: {df.isna().sum().sum()}")

def create_time_based_features(df):
    """Create time-based features with cyclic encoding."""
    if not pd.api.types.is_datetime64_any_dtype(df['Order Date']):
        df['Order Date'] = pd.to_datetime(df['Order Date'], errors='coerce')

    df['Order Date'] = df['Order Date'].fillna(pd.Timestamp('2000-01-01'))  

    features_df = pd.DataFrame(index=df.index)

    # Cyclic encoding for Day_of_Week and Month
    features_df['Day_of_Week_sin'] = np.sin(2 * np.pi * df['Order Date'].dt.dayofweek / 7)
    features_df['Day_of_Week_cos'] = np.cos(2 * np.pi * df['Order Date'].dt.dayofweek / 7)
    features_df['Month_sin'] = np.sin(2 * np.pi * df['Order Date'].dt.month / 12)
    features_df['Month_cos'] = np.cos(2 * np.pi * df['Order Date'].dt.month / 12)

    # Add Is_Weekend (binary feature)
    features_df['Is_Weekend'] = df['Order Date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

    check_missing_values(features_df, "Time-Based Features")
    return features_df


def create_lag_features(df, features_df):
    """Create lag features and add to the existing features_df."""
    
    features_df['Sales_Lag_7'] = df['Sales'].shift(7)

    check_missing_values(features_df, "Lag Features")
    return features_df


def create_rolling_features(df, features_df):
    """Create rolling window statistics for each category."""
    categories = df['Category'].unique()

    for category in categories:
        cat_df = df[df['Category'] == category]

        rolling_sales = cat_df['Sales'].rolling(window=7, min_periods=7).mean()
        rolling_profit = cat_df['Profit'].rolling(window=7, min_periods=7).mean()

        features_df.loc[cat_df.index, f'Rolling_Sales_7_{category}'] = rolling_sales
        features_df.loc[cat_df.index, f'Rolling_Profit_7_{category}'] = rolling_profit

    for col in features_df.columns:
        if 'Rolling_Sales_7' in col:
            features_df[col] = features_df[col].ffill()  
        elif 'Rolling_Profit_7' in col:
            features_df[col] = features_df[col].bfill()  

    check_missing_values(features_df, "Rolling Features")
    return features_df


def create_cumulative_features(df, features_df):
    """Create cumulative features."""
    features_df['Cumulative_Sales'] = df.groupby('Category')['Sales'].cumsum()
    features_df['Cumulative_Profit'] = df.groupby('Category')['Profit'].cumsum()
    
    check_missing_values(features_df, "Cumulative Features")
    return features_df


def create_category_level_features(df, features_df):
    """Create category-level features."""
    category_sales_avg = df.groupby('Category')['Sales'].transform('mean')
    
    features_df['Sales_vs_Category_Avg'] = df['Sales'] - category_sales_avg
    
    check_missing_values(features_df, "Category-Level Features")
    return features_df


def create_growth_and_extra_features(df, features_df):
    """Create growth and extra features."""
    features_df['Category_Sales_Growth'] = df.groupby('Category')['Sales'].pct_change()
    features_df['Rolling_30_Sales'] = df.groupby('Category')['Sales'].rolling(30).mean().reset_index(level=0, drop=True)
    
    check_missing_values(features_df, "Growth and Extra Features")
    return features_df


def create_interaction_features(df, features_df):
    """Create interaction features between time-based and categorical/statistical features."""
    features_df['Weekend_Sales'] = features_df['Is_Weekend'] * df['Sales']
    
    
    check_missing_values(features_df, "Interaction Features")
    return features_df


def handle_missing_values(df, features_df):
    """Handle missing values created by lagged features, rolling statistics, and growth/extra features."""
    # Dynamically identify columns with NaN values
    nan_columns = features_df.columns[features_df.isna().any()].tolist()
    
    # Fill NaN values based on column type or name
    for col in nan_columns:
        if 'Rolling' in col:  # Handle rolling features
            if 'Sales' in col:
                features_df[col] = features_df[col].fillna(df['Sales'].mean())
            elif 'Profit' in col:
                features_df[col] = features_df[col].fillna(df['Profit'].mean())
            else:
                features_df[col] = features_df[col].fillna(0)  # Default to 0 for other rolling features
        elif 'Lag' in col:  # Handle lag features
            features_df[col] = features_df[col].fillna(df['Sales'].mean())
        elif 'Growth' in col:  # Handle growth features
            features_df[col] = features_df[col].fillna(0)  # Default to 0 for growth features
        else:  # Handle all other columns
            features_df[col] = features_df[col].fillna(0)  # Default to 0 for unknown columns
    
    # Re-check for missing values after handling
    check_missing_values(features_df, "Missing Value Handling")
    
    return features_df


# Call the function to create time-based features
features_df = create_time_based_features(cleaned_df)

# Call the function to create lag features and add them to the same features_df
features_df = create_lag_features(cleaned_df, features_df)

# Call the function to create rolling features and add them to the same features_df
features_df = create_rolling_features(cleaned_df, features_df)

# Call the function to create cumulative features and add them to the same features_df
features_df = create_cumulative_features(cleaned_df, features_df)

# Call the function to create category-level features and add them to the same features_df
features_df = create_category_level_features(cleaned_df, features_df)

# Call the function to create growth and extra features and add them to the same features_df
features_df = create_growth_and_extra_features(cleaned_df, features_df)

# Call the function to create interaction features and add them to the same features_df
features_df = create_interaction_features(cleaned_df, features_df)

# Call the function to handle missing values
features_df = handle_missing_values(cleaned_df, features_df)

# Print the first few rows to check the result
print(features_df.head(10))
print(features_df.columns.tolist())
print(cleaned_df.columns.tolist())

# Combine the original dataframe with the engineered features
df_combined = pd.concat([cleaned_df, features_df], axis=1)

# Optionally, reset the index if needed
df_combined.reset_index(drop=True, inplace=True)

print(f"Total missing values in the combined dataset: {df_combined.isna().sum().sum()}")
print(f"Shape of the combined dataset: {df_combined.shape}")

Missing values in Time-Based Features: 0
Missing values in Lag Features: 7
Missing values in Rolling Features: 93
Missing values in Cumulative Features: 93
Missing values in Category-Level Features: 93
Missing values in Growth and Extra Features: 183
Missing values in Interaction Features: 183
Missing values in Missing Value Handling: 0
   Day_of_Week_sin  Day_of_Week_cos  ...  Rolling_30_Sales  Weekend_Sales
0        -0.974928        -0.222521  ...        172.398334         66.120
1        -0.974928        -0.222521  ...        172.398334        120.366
2        -0.974928        -0.222521  ...        172.398334         44.865
3        -0.974928        -0.222521  ...        172.398334         55.242
4        -0.974928        -0.222521  ...        172.398334        113.670
5        -0.974928        -0.222521  ...        172.398334        408.300
6        -0.781831         0.623490  ...        172.398334        314.220
7         0.000000         1.000000  ...        172.398334          0

In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

def encode_data(df_combined):
    # Initialize the OneHotEncoder
    onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)

    # Initialize the LabelEncoder for the 'Segment' column
    label_encoder = LabelEncoder()

    # Apply One-Hot Encoding to the 'Category' column if it exists
    if 'Category' in df_combined.columns:
        category_encoder = OneHotEncoder(sparse_output=False)
        encoded_category = category_encoder.fit_transform(df_combined[['Category']])
        encoded_category_df = pd.DataFrame(encoded_category, columns=category_encoder.get_feature_names_out(['Category']))
        
        # Drop the original 'Category' column after encoding
        df_combined.drop(columns=['Category'], inplace=True)
        
        # Concatenate the encoded 'Category' columns back to the dataframe
        df_combined = pd.concat([df_combined, encoded_category_df], axis=1)

    # Label Encode 'Segment' column
    if 'Segment' in df_combined.columns:
        df_combined['Segment'] = label_encoder.fit_transform(df_combined['Segment'])

    # Apply Priority Mapping to 'Order Priority' column
    if 'Order Priority' in df_combined.columns:
        priority_mapping = {"low": 1, "medium": 2, "high": 3, "critical": 4}
        df_combined['Order Priority_Code'] = df_combined['Order Priority'].map(priority_mapping)
        df_combined.drop('Order Priority', axis=1, inplace=True)  # Drop original Order Priority column in place

    # Return the newly encoded DataFrame
    return df_combined

# Example usage
# Load your dataframe here
# df_combined = pd.read_csv('your_data.csv')
df_combined = encode_data(df_combined)
print(df_combined.head())

  Order Date  Segment  ...  Category_Technology  Order Priority_Code
0 2011-01-01        0  ...                  0.0                    3
1 2011-01-01        0  ...                  0.0                    2
2 2011-01-01        2  ...                  0.0                    3
3 2011-01-01        0  ...                  0.0                    2
4 2011-01-01        0  ...                  0.0                    2

[5 rows x 30 columns]


In [21]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

df_processed = df_combined.copy()

Total_null = df_processed.isnull().sum().sum()
print("The total number of Null in the data :" , Total_null)

order_date_stored = df_processed['Order Date'].copy()
feature_name_stored = df_processed.columns.tolist()
df_processed = df_processed.drop(columns=['Order Date'])

print("Feature_Names : ", feature_name_stored)

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df_processed), columns = df_processed.columns, index = df_processed.index)
#print(df_normalized.head())


time_steps = 30

def create_sequence(data, time_steps = 30):
    X, y = [],[]
    for i in range(len(data) - time_steps):
        X.append(data.iloc[i:i+time_steps].values)
        y.append(data.iloc[i+time_steps][target_column])
    return np.array(X), np.array(y)

target_column = 'Sales'

X,y = create_sequence(df_normalized, time_steps)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


print(X_train[:5])
print("Shape of the X_train: ",X_train.shape)
print("Shape of the X_test:",X_test.shape)
print("Shape of the y_train: ",y_train.shape)
print("Shape of the y_test",y_test.shape)

np.save('X_train.npy',X_train)
np.save('X_test.npy',X_test)
np.save('y_train.npy',y_train)
np.save('y_test.npy',y_test)

The total number of Null in the data : 0
Feature_Names :  ['Order Date', 'Segment', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost', 'Unit_Price', 'Day_of_Week_sin', 'Day_of_Week_cos', 'Month_sin', 'Month_cos', 'Is_Weekend', 'Sales_Lag_7', 'Rolling_Sales_7_Office Supplies', 'Rolling_Profit_7_Office Supplies', 'Rolling_Sales_7_Furniture', 'Rolling_Profit_7_Furniture', 'Rolling_Sales_7_Technology', 'Rolling_Profit_7_Technology', 'Cumulative_Sales', 'Cumulative_Profit', 'Sales_vs_Category_Avg', 'Category_Sales_Growth', 'Rolling_30_Sales', 'Weekend_Sales', 'Category_Furniture', 'Category_Office Supplies', 'Category_Technology', 'Order Priority_Code']
[[[0.         0.11301313 0.35294118 ... 1.         0.         0.66666667]
  [0.         0.20635789 0.23529412 ... 1.         0.         0.33333333]
  [1.         0.07643822 0.23529412 ... 1.         0.         0.66666667]
  ...
  [0.5        0.02850624 0.11764706 ... 1.         0.         0.33333333]
  [0.         0.02753916 0.11764